In [10]:
import pymysql, talib, sys, os
import pandas as pd , numpy as np
#################################
class HKStock:
    conn = None
    symbol=None
    daily=None
    ####################
    def __init__(self,symbol):
        self.symbol = symbol
        self.populate()
    ####################
    def populate(self):
        if self.conn is None:
            self.conn = pymysql.connect(host='tptcn.ddns.net', port=53306, db='p_master',
                                   user=os.environ['MYSQL_READUSER'], password=os.environ['MYSQL_READPASSWORD'],
                                   cursorclass=pymysql.cursors.DictCursor)
        tmpl = "SELECT  * FROM `consolidated_daily` WHERE `symbol`='%s' "
        sql = tmpl % (self.symbol)
        self.daily= pd.read_sql(sql, self.conn, index_col=['Date'] )
        ### temp
        sOpen  = self.daily['Open']
        sHigh  = self.daily['High']
        sLow   = self.daily['Low']
        sClose = self.daily['Close']
        sVolume= self.daily['Volume']
        ### candle
        #self.daily['CDLLONGLINE'] =  talib.CDLLONGLINE(sOpen, sHigh, sLow, sClose)
        #self.daily['CDL2CROWS'] =  talib.CDL2CROWS(sOpen, sHigh, sLow, sClose)
        #self.daily['CDLIDENTICAL3CROWS'] =  talib.CDLIDENTICAL3CROWS(sOpen, sHigh, sLow, sClose)
        #self.daily['CDLTAKURI'] =  talib.CDLTAKURI(sOpen, sHigh, sLow, sClose)
        ### TA
        #self.daily['RSI14'] = talib.RSI(sClose, timeperiod=14)
        #self.daily['SMA50'] = talib.SMA(sClose, timeperiod=50)
        #self.daily['OBV'] = talib.OBV(sClose , sVolume)
        self.daily['BBuband'], self.daily['BBmband'], self.daily['BBlband'] = talib.BBANDS(sClose, timeperiod=20)
        ### Label
        theMax=self.daily['VWAP']
        theMin=self.daily['VWAP']
        for n in range(1,11):
            theMax = np.maximum(theMax , self.daily['VWAP'].shift(0 - n) )
            theMin = np.minimum(theMin , self.daily['VWAP'].shift(0 - n) )
        self.daily['LabelMax'] = theMax.diff()
        self.daily['LabelMin'] = theMin.diff()
        
        #self.daily['LabelPositive'] = (theMax / self.daily['VWAP']) >= 1.05
        #self.daily['LabelNegative'] = (theMin / self.daily['VWAP']) <= 0.95
        
        #self.daily['LabelOutSideBBand'] = np.logical_or(np.greater_equal(self.daily['VWAP'], self.daily['BBuband']) , np.less_equal(self.daily['VWAP'], self.daily['BBlband']))
            
    ####################
    def __del__(self):
        if self.conn is not None:
            self.conn.close()
            self.conn = None
#############################################
df = HKStock('0388.HK').daily


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in maximum
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in minimum


In [12]:
#print df.tail()
#print
#df[ df['LabelNegative'] ].dropna()
df.tail(120)


,symbol,Open,High,Low,Close,Volume,Turnover,ShortVolume,ShortTurnover,VWAP,VWAPampre,VWAPam,VWAPpm,VWAPpmpost,BBuband,BBmband,BBlband,LabelMax,LabelMin
Date,,,,,,,,,,,,,,,,,,,
2018-07-04,0388.HK,232.000,234.0,227.6,229.6,5944471,1362114058,562500.0,128941240.0,229.140,233.199,229.619,228.749,229.600,270.460895,247.51,224.559105,0.000,0.000
2018-07-05,0388.HK,229.800,231.0,225.2,227.4,6536464,1485464415,152700.0,34621900.0,227.258,230.122,229.040,226.553,227.400,269.985299,245.98,221.974701,0.000,0.000
2018-07-06,0388.HK,227.285,230.6,225.4,228.8,6235965,1423024613,279200.0,63587620.0,228.196,228.023,227.059,229.104,228.800,268.873429,244.49,220.106571,0.000,0.938
2018-07-09,0388.HK,228.000,234.6,230.6,233.0,5721784,1335867865,526100.0,122960180.0,233.471,232.494,233.227,233.748,233.000,266.212126,242.97,219.727874,0.000,2.746
2018-07-10,0388.HK,231.000,237.6,234.0,234.0,5017550,1181024752,251000.0,59182340.0,235.379,233.998,235.943,235.088,234.000,263.840350,241.67,219.499650,1.390,0.000
2018-07-11,0388.HK,234.860,233.0,229.4,231.8,4072204,940444675,193900.0,44796700.0,230.942,231.549,231.085,230.674,231.800,260.841705,240.20,219.558295,2.341,0.000
2018-07-12,0388.HK,231.800,235.0,231.4,233.4,3571513,833699124,338500.0,79048600.0,233.430,231.783,232.998,233.774,233.400,257.406034,238.84,220.273966,0.000,1.054
2018-07-13,0388.HK,233.400,236.8,233.8,234.2,3506651,823570010,350200.0,82217440.0,234.859,236.557,235.295,234.529,234.200,254.196960,237.67,221.143040,0.000,0.000
2018-07-16,0388.HK,236.731,236.0,231.6,233.8,1973291,460803308,210500.0,49210980.0,233.520,236.274,233.132,233.804,233.800,250.970497,236.58,222.189503,0.000,0.000
